### Gloden cross strategy

In [1]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import csv

In [2]:
data = pd.read_csv('AUDUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
16010,2022-06-30 16:45:00+01:00,0.689669,0.690751,0.689669,0.690632,0.690632,0
16011,2022-06-30 17:00:00+01:00,0.690632,0.690655,0.690222,0.690398,0.690398,0
16012,2022-06-30 17:15:00+01:00,0.690651,0.691706,0.690498,0.691520,0.691520,0
16013,2022-06-30 17:30:00+01:00,0.691659,0.692022,0.690851,0.691052,0.691052,0
16014,2022-06-30 17:45:00+01:00,0.691310,0.691706,0.691118,0.691281,0.691281,0


In [3]:
df = data.copy()
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.tz_localize(None)
df

,timestamp,open,high,low,close,adj close,volumn
0,2021-11-04 17:00:00,0.739776,0.739973,0.739536,0.739919,0.739919,0
1,2021-11-04 17:15:00,0.739793,0.740192,0.739661,0.739776,0.739776,0
2,2021-11-04 17:30:00,0.739826,0.739973,0.739486,0.739919,0.739919,0
3,2021-11-04 17:45:00,0.739722,0.739973,0.739322,0.739656,0.739656,0
4,2021-11-04 18:00:00,0.739738,0.739919,0.739497,0.739864,0.739864,0
...,...,...,...,...,...,...,...
16010,2022-06-30 15:45:00,0.689669,0.690751,0.689669,0.690632,0.690632,0
16011,2022-06-30 16:00:00,0.690632,0.690655,0.690222,0.690398,0.690398,0
16012,2022-06-30 16:15:00,0.690651,0.691706,0.690498,0.691520,0.691520,0
16013,2022-06-30 16:30:00,0.691659,0.692022,0.690851,0.691052,0.691052,0


### Create strategy

In [19]:
percent_r = ta.Strategy(
    name = 'Willams % R',
    description = 'Willams % R Strategy',
    ta = [
        {'kind': 'ema', 'length': 100},
        {'kind': 'willr', 'length': 20},
        {'kind': 'atr', 'length': 14}
    ]
)
df.ta.strategy(percent_r)

In [20]:
df

,timestamp,open,high,low,close,adj close,volumn,EMA_100,WILLR_20,ATRr_14
0,2021-11-04 17:00:00+00:00,0.739776,0.739973,0.739536,0.739919,0.739919,0,NaN,NaN,NaN
1,2021-11-04 17:15:00+00:00,0.739793,0.740192,0.739661,0.739776,0.739776,0,NaN,NaN,NaN
2,2021-11-04 17:30:00+00:00,0.739826,0.739973,0.739486,0.739919,0.739919,0,NaN,NaN,NaN
3,2021-11-04 17:45:00+00:00,0.739722,0.739973,0.739322,0.739656,0.739656,0,NaN,NaN,NaN
4,2021-11-04 18:00:00+00:00,0.739738,0.739919,0.739497,0.739864,0.739864,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11930,2022-04-29 16:45:00+01:00,0.710424,0.711288,0.710252,0.710308,0.710308,0,0.713454,-89.615701,0.001238
11931,2022-04-29 17:00:00+01:00,0.710252,0.710252,0.709084,0.709210,0.709210,0,0.713370,-98.604430,0.001237
11932,2022-04-29 17:15:00+01:00,0.709144,0.709824,0.709144,0.709310,0.709310,0,0.713289,-97.487444,0.001197
11933,2022-04-29 17:30:00+01:00,0.709215,0.709320,0.708843,0.709044,0.709044,0,0.713205,-97.653728,0.001146


### Create signal¶

In [4]:
class BacktestWilliamsR:
    def __init__(self, params):
        self.willr_status = 'idle' # idle, overbought, oversold
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.willr = params['willr']
        
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        
        self.pip_value = params['pip_value']
        self.target_tp = params['target_tp']
        self.rr = params['rr']
        self.df = params['df']
        
        self.latest_sl = 0
        self.latest_tp = 0
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
    
    def calLotSize(self, entry, exit):
        return abs(round(self.target_tp / (entry - exit)  , 2))
#       return abs(round(self.target_tp / (((entry - exit) / self.pip ) * self.pip_value ) , 2))

    def longCondition(self, row):
        if self.willr_status == 'idle':
            if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
                self.willr_status = 'oversold'
                        
            elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
                self.willr_status = 'overbought'
                        
        elif self.willr_status == 'oversold':
            if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] > -50):
                row['entries'] = True
                self.current_action = 'long'
                
                self.latest_sl = row['open'] - row['ATRr_'+str(self.atr)]
                self.latest_tp = row['open'] + (self.rr * abs(row['open'] - self.latest_sl))
                row['lot_size'] = self.calLotSize(row['open'], self.latest_tp)        
                self.willr_status = 'idle'
                        
            elif (row['EMA_'+str(self.ema)] > row['close']):
                self.willr_status = 'idle'
                
    def longResult(self, row):
        if row['low'] <= self.latest_sl:
            self.current_action = 'close'
            row['exits'] = True
            row['price_action'] = self.latest_sl
            row['lot_size'] = 0
                    
        elif row['high'] >= self.latest_tp:
            self.current_action = 'close'
            row['exits'] = True
            row['price_action'] = self.latest_tp
            row['lot_size'] = 0
        
    def shortCondition(self, row):
        if self.willr_status == 'idle':
            if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
                self.willr_status = 'oversold'
                        
            elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
                self.willr_status = 'overbought'
                
        elif self.willr_status == 'overbought':
            if (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] < -50):
                row['short_entries'] = True
                self.current_action = 'short'
                
                self.latest_sl = row['open'] + row['ATRr_'+str(self.atr)]
                self.latest_tp = row['open'] - (self.rr * abs(row['open'] - self.latest_sl))
                row['lot_size'] = self.calLotSize(row['open'], self.latest_tp)        
                self.willr_status = 'idle'
                        
            elif (row['EMA_'+str(self.ema)] < row['close']):
                self.willr_status = 'idle'
                
    def shortResult(self, row):
        if row['high'] >= self.latest_sl:
            self.current_action = 'close'
            row['short_exits'] = True
            row['price_action'] = self.latest_sl
            row['lot_size'] = 0
                    
        elif row['low'] <= self.latest_tp:
            self.current_action = 'close'
            row['short_exits'] = True
            row['price_action'] = self.latest_tp
            row['lot_size'] = 0
                    
    def createLongOrders(self, row):
        if self.current_action == 'close':
            self.longCondition(row)
        else:
            self.longResult(row)
        
        return row
    
    def createShortOrders(self, row):
        if self.current_action == 'close':
            self.shortCondition(row)
        else:
            self.shortResult(row)
                    
        return row
    
    def createBothOrders(self, row):
        if self.current_action == 'close':
            self.longCondition(row)
            self.shortCondition(row)
            
        elif self.current_action == 'long':
            self.longResult(row)
                    
        elif self.current_action == 'short':
            self.shortResult(row)
                    
        return row
        
    def createStrategy(self):
        percent_r = ta.Strategy(
            name = 'Willams % R',
            description = 'Willams % R Strategy',
            ta = [
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'willr', 'length': self.willr},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(percent_r)
        
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['entries'] = False
        self.df['exits'] = False
        
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['open']
        
        self.df['lot_size'] = np.nan
        
    def getBothOrders(self):
        self.createStrategy()
        self.df = self.df.apply(lambda row : self.createBothOrders(row), axis=1)
                                                                
        return self.df
        
    def getLongOrders(self):
        self.createStrategy()
        self.df = self.df.apply(lambda row : self.createLongOrders(row), axis=1)
                                                                
        return self.df
    
    def getShortOrders(self):
        self.createStrategy()
        self.df = self.df.apply(lambda row : self.createShortOrders(row), axis=1)
                                                                
        return self.df

#     def backtest(self):
#         n_bars = 3
#         percent_r = ta.Strategy(
#             name = 'Willams % R',
#             description = 'Willams % R Strategy',
#             ta = [
#                 {'kind': 'ema', 'length': self.ema},
#                 {'kind': 'willr', 'length': self.willr},
#                 {'kind': 'atr', 'length': self.atr}
#             ]
#         )
#         self.df.ta.strategy(percent_r)
#         self.df = self.df.copy().dropna()
#         self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
#         self.df['action'] = None
        
#         self.df['entries'] = False
#         self.df['exits'] = False
#         self.df['short_entries'] = False
#         self.df['short_exits'] = False
#         self.df['price_action'] = self.df['close']
        
#         for index, row in self.df.iterrows():
#             entry = 0
            
#             if self.current_action == 'close':
                
#                 if self.willr_status == 'idle':
#                     if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
#                         self.willr_status = 'oversold'
                        
#                     elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
#                         self.willr_status = 'overbought'
                        
#                 elif self.willr_status == 'oversold':
#                     if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] > -50):
#                         self.df.at[index, 'action'] = 'buy'
#                         self.df.at[index, 'entries'] = True
#                         self.df.at[index, 'price_action'] = row['open']
#                         self.order['action'] = 'buy'
                                
#                         self.order['open time'] = row['timestamp']
#                         self.order['open'] = row['close']
#                         self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
#                         self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
#                         self.current_action = 'buy'
                        
#                         self.willr_status = 'idle'
                        
#                     elif (row['EMA_'+str(self.ema)] > row['close']):
#                         self.willr_status = 'idle'
                        
#                 elif self.willr_status == 'overbought':
#                     if (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] < -50):
#                         self.df.at[index, 'action'] = 'sell'
#                         self.df.at[index, 'short_entries'] = True
#                         self.df.at[index, 'price_action'] = row['open']
#                         self.order['action'] = 'sell'
                                
#                         self.order['open time'] = row['timestamp']
#                         self.order['open'] = row['close']
#                         self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
#                         self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
#                         self.current_action = 'sell'
                        
#                         self.willr_status = 'idle'
                        
#                     elif (row['EMA_'+str(self.ema)] < row['close']):
#                         self.willr_status = 'idle'
        
#             if self.current_action == 'buy':
#                 if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
#                     self.unknow_result.append(row)
                    
#                 if row['low'] <= self.order['S/L']:
#                     self.df.at[index, 'exits'] = True
#                     self.df.at[index, 'price_action'] = row['low']
                    
#                     self.order['result'] = 'S/L'
#                     self.order['close time'] = row['timestamp']
#                     self.order['close'] = self.order['S/L']
                    
#                     self.current_action = 'close'
#                     self.orders = self.orders.append(self.order, ignore_index = True)
#                     if self.plot == True: self.plot_order(self.order)
#                     self.reset_order()
                    
#                 elif row['high'] >= self.order['T/P']:
#                     self.df.at[index, 'exits'] = True
#                     self.df.at[index, 'price_action'] = row['high']
                    
#                     self.order['result'] = 'T/P'
#                     self.order['close time'] = row['timestamp']
#                     self.order['close'] = self.order['T/P']
                    
#                     self.current_action = 'close'
#                     self.orders = self.orders.append(self.order, ignore_index = True)
#                     if self.plot == True: self.plot_order(self.order)
#                     self.reset_order()

#             elif self.current_action == 'sell':
#                 if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
#                     self.unknow_result.append(row)
                    
#                 if row['high'] >= self.order['S/L']:
#                     self.df.at[index, 'short_exits'] = True
#                     self.df.at[index, 'price_action'] = row['high']
                    
#                     self.order['result'] = 'S/L'
#                     self.order['close time'] = row['timestamp']
#                     self.order['close'] = self.order['S/L']
                    
#                     self.current_action = 'close'
#                     self.orders = self.orders.append(self.order, ignore_index = True)
#                     if self.plot == True: self.plot_order(self.order)
#                     self.reset_order()
                    
#                 elif row['low'] <= self.order['T/P']:
#                     self.df.at[index, 'short_exits'] = True
#                     self.df.at[index, 'price_action'] = row['low']
                    
#                     self.order['result'] = 'T/P'
#                     self.order['close time'] = row['timestamp']
#                     self.order['close'] = self.order['T/P']
                    
#                     self.current_action = 'close'
#                     self.orders = self.orders.append(self.order, ignore_index = True)
#                     if self.plot == True: self.plot_order(self.order)
#                     self.reset_order()

#         try:
#             win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
#             loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

#             win_orders = int((win_rate * len(self.orders)) / 100 )
#             loss_orders = len(self.orders) - win_orders
#             gain = (win_orders * self.rr) - loss_orders

#             result_info = f"total orders: {len(self.orders)}\nema: {self.ema}, willr: {self.willr}, atr: {self.atr}, overbought: {self.overbought}, oversold: {self.oversold}, gain: {gain}\n"
#             return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
#         except:
#     #         print("No orders to action")
#             return {'win_rate': 0, 'info': 'No orders to action'}

In [8]:
pair_test = BacktestWilliamsR({
    'atr': 14,
    'ema': 100,
    'willr': 20,
    'overbought': -20,
    'oversold': -80,
    'pip': 0.0001,
    'pip_value': 10,
    'rr': 1.5,
    'target_tp': 10,
    'df': df.copy()
})

result_long = pair_test.getLongOrders().set_index('timestamp')
result_short = pair_test.getShortOrders().set_index('timestamp')
result_both = pair_test.getBothOrders().set_index('timestamp')
result_both

,open,high,low,close,adj close,volumn,EMA_100,WILLR_20,ATRr_14,entries,exits,short_entries,short_exits,price_action,lot_size
timestamp,,,,,,,,,,,,,,,
2021-11-04 17:00:00,0.739776,0.739973,0.739536,0.739919,0.739919,0,NaN,NaN,NaN,False,False,False,False,0.739776,NaN
2021-11-04 17:15:00,0.739793,0.740192,0.739661,0.739776,0.739776,0,NaN,NaN,NaN,False,False,False,False,0.739793,NaN
2021-11-04 17:30:00,0.739826,0.739973,0.739486,0.739919,0.739919,0,NaN,NaN,NaN,False,False,False,False,0.739826,NaN
2021-11-04 17:45:00,0.739722,0.739973,0.739322,0.739656,0.739656,0,NaN,NaN,NaN,False,False,False,False,0.739722,NaN
2021-11-04 18:00:00,0.739738,0.739919,0.739497,0.739864,0.739864,0,NaN,NaN,NaN,False,False,False,False,0.739738,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 15:45:00,0.689669,0.690751,0.689669,0.690632,0.690632,0,0.688905,-21.225178,0.001101,False,False,False,False,0.689669,NaN
2022-06-30 16:00:00,0.690632,0.690655,0.690222,0.690398,0.690398,0,0.688934,-26.191532,0.001053,False,False,False,False,0.690632,NaN
2022-06-30 16:15:00,0.690651,0.691706,0.690498,0.691520,0.691520,0,0.688985,-3.901740,0.001072,False,False,False,False,0.690651,NaN


In [5]:
"""
Grid parameter example
willr: 10 -> 40
overbought oversold: 70->90, 20->30
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            params.append([i, j])
    return params
params_list = create_grid_params(10, 40, 10, 30)
len(params_list)

651

In [24]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 651
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BacktestWilliamsR({'atr': 14, 'ema': 100,  'willr': param[0],'overbought': -1*param[1], 'oversold': -1*(100-param[1]), 'pip_value': 0.001, 'rr': 1.5, 'df': df.copy(), 'plot': False})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 300:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 651 to 651
win rate: 68.2 %
total orders: 305
ema: 100, willr: 12, atr: 14, overbought: -29, oversold: -71, gain: 215.0



### ETHUSDT 15m
* win rate: 79.69 %
* total orders: 256
* ema: 100, willr: 37, atr: 14, overbought: -14, oversold: -86, gain: 254.0

### GBPUSD 15m
* win rate: 73.28 %
* total orders: 131
* ema: 100, willr: 17, atr: 14, overbought: -15, oversold: -85, gain: 106.5

* win rate: 64.33 %
* total orders: 342
* ema: 100, willr: 10, atr: 14, overbought: -29, oversold: -71, gain: 208.0

### USDJPY 15m

* win rate: 65.38 %
* total orders: 130
* ema: 100, willr: 12, atr: 14, overbought: -17, oversold: -83, gain: 80.0

### AUDUSD 15m
* win rate: 76.92 %
* total orders: 130
* ema: 100, willr: 29, atr: 14, overbought: -30, oversold: -70, gain: 117.5

### EURUSD 15m
* win rate: 69.29 %
* total orders: 127
* ema: 100, willr: 24, atr: 14, overbought: -25, oversold: -75, gain: 90.5

* win rate: 61.84 %
* total orders: 304
* ema: 100, willr: 12, atr: 14, overbought: -30, oversold: -70, gain: 163.5

In [6]:
good_params = {'win_long': 0, 'info_long': '', 'win_short': 0, 'info_short': ''}
start = 0
end = len(params_list)
win_long = 0
win_short = 0

loop_num = start
for param in params_list[start:end]:
    pair_test = BacktestWilliamsR({
        'atr': 14,
        'ema': 100,
        'willr': param[0],
        'overbought': -1*param[1],
        'oversold': -1*(100-param[1]),
        'pip': 0.0001,
        'pip_value': 10,
        'rr': 1.5,
        'target_tp': 10,
        'df': df.copy()
    })
    result_long = pair_test.getLongOrders().set_index('timestamp')
    result_short = pair_test.getShortOrders().set_index('timestamp')
    
    port_long = vbt.Portfolio.from_signals(result_long.price_action,
                                      entries=result_long.entries.astype('bool'),
                                      exits=result_long.exits.astype('bool'),
                                      size=result_long.lot_size, 
                                      freq="15m",
                                      init_cash=20000,
    #                                   fees=0.0005, 
                                      slippage=0.0002
                                     )    

    port_short = vbt.Portfolio.from_signals(result_short.price_action,
                                      short_entries=result_short.short_entries.astype('bool'),
                                      short_exits=result_short.short_exits.astype('bool'),
                                      size=result_short.lot_size,
                                      size_type=0,
                                      freq="15m",
                                      init_cash=20000,
    #                                   fees=0.0005, 
                                      slippage=0.0002
                                     )
    
    stats_long = port_long.stats()
    stats_short = port_short.stats()
    
    
#     result = backtest_rsi_o2(df, ema_len=param[0], rsi_len=param[1], atr_len=param[2], overbought=param[3], oversold=param[4], rr=param[5])
    if stats_long['Win Rate [%]'] > win_long and stats_long['Total Trades'] > 200:
        win_long = stats_long['Win Rate [%]']
        good_params['win_long'] = stats_long['Win Rate [%]']
        good_params['info_long'] = {'Max Drawdown % ': stats_long["Max Drawdown [%]"], 
                                    'atr': 14,
                                    'ema': 100,
                                    'willr': param[0],
                                    'overbought': -1*param[1],
                                    'oversold': -1*(100-param[1])
                                   }
        
    if stats_short['Win Rate [%]'] > win_short and stats_short['Total Trades'] > 200:
        win_short = stats_short['Win Rate [%]']
        good_params['win_short'] = stats_short['Win Rate [%]']
        good_params['info_short'] = {'Max Drawdown % ': stats_short["Max Drawdown [%]"], 
                                    'atr': 14,
                                    'ema': 100,
                                    'willr': param[0],
                                    'overbought': -1*param[1],
                                    'oversold': -1*(100-param[1])
                                   }
        
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin long: {good_params['win_long']} %\n{good_params['info_long']} \nwin short: {good_params['win_short']} %\n{good_params['info_short']} ")

run: 651 to 651
win long: 0 %
 
win short: 0 %
 
